# Dimensions of the CSD computation

In [1]:
import json
import time
import pandas as pd
import src.paths
from src.data import load_ephys
from src.lfp import remove_dc_offset, combine_neighbors, decimate_by_2
from src.lfp import bandpass_1_300Hz, notch_60Hz, compute_csd

In [2]:
date = '2022-07-27'
region = 'hpc'

In [3]:
with open(src.paths.REF / 'date_recoffset.json', 'r') as f:
    offsets = json.load(f)

In [4]:
path = '/mnt/home/larend/kfx/data/ephys/hpc/lf/2022-07-27-07-41-00_M015_SAL_PFC_HPC_0_0_0mpk_g0_t0.imec1.lf.bin'

In [5]:
lfpstart, lfpend = offsets[date]
duration = (lfpend - lfpstart) / 2500 / 60

In [6]:
print(f'date: {date}')
print(f'reading path: {path}')
print(f'lfpstart, lfpend:', lfpstart, lfpend)
print(f'duration: {duration} min')

date: 2022-07-27
reading path: /mnt/home/larend/kfx/data/ephys/hpc/lf/2022-07-27-07-41-00_M015_SAL_PFC_HPC_0_0_0mpk_g0_t0.imec1.lf.bin
lfpstart, lfpend: 24077 18029080
duration: 120.03335333333332 min


In [7]:
intervals = []
startmin = 0
while True:
    endmin = min(startmin + 1, duration)
    intervals.append((startmin, endmin))
    if endmin == duration:
        break
    startmin = endmin
print(f'# batches:', len(intervals))

# batches: 121


In [8]:
intervals[20]

(20, 21)

In [9]:
start, end = 20, 21
raw = load_ephys(path, startmin=start, endmin=end, offset=lfpstart, cleaned=True)
X = raw.values

In [10]:
X = remove_dc_offset(X)
X = combine_neighbors(X)
X = bandpass_1_300Hz(X)
X = notch_60Hz(X)
X = decimate_by_2(X)

In [11]:
import quantities as pq
from src.icsd import SplineiCSD

In [12]:
h = 20 * 1e-6 * pq.m             # inter-electrode spacing [m]
d = 0.5 * 1e-3 * pq.m            # electrode diameter [m]
sigma = 0.3 * pq.S / pq.m        # intracellular medium conductivity [S/m]
sigma_top = 0.3 * pq.S / pq.m    # cortical surface conductivity [S/m]
z = np.arange(X.shape[1]) * h    # electrode coordinates [m]
X = X.T * pq.V                   # recorded potential [V]
estimator = SplineiCSD(
    X, coord_electrode=z, diam=d, sigma=sigma, sigma_top=sigma_top,
    tol=1e-12, f_type='gaussian', f_order=(20, 5), num_steps=len(z)
)
csd = estimator.get_csd()
csd = estimator.filter_csd(csd)  # apply spatial smoothing
csd = csd.T

In [13]:
csd.shape

(75000, 192)

In [17]:
csd.dimensionality

Dimensionality({meter: -3, ampere: 1})

In [20]:
csd.__dict__

{'_dimensionality': Dimensionality({meter: -3, ampere: 1})}